In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings # To suppress warnings from OPTICS

from sklearn.cluster import DBSCAN, KMeans, OPTICS

import colorsys

# Obj file clustering

In [2]:
# DBSCAN with color (works with 4D grayscale and 6D color)
def dbscan_color(point_cloud, pc_colors, eps, mp, debug=False):
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        
        clustering = DBSCAN(eps=eps, min_samples=mp, n_jobs=-1).fit(pc_colors)
        labels = clustering.labels_
        print("clustering: ", clustering)
        print("labels: ", labels)

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    print("colors: ", colors)
    colors[labels < 0] = 0
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

    return point_cloud, labels


# DBSCAN without color
def dbscan(point_cloud, eps, mp, debug=False):

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            point_cloud.cluster_dbscan(eps=eps, min_points=mp, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

    return point_cloud, labels

def optics_color(point_cloud, pc_colors, eps, mp, debug=False):
    
    point_cloud = np.asarray(point_cloud.points)
    clustering = OPTICS(min_samples=mp, max_eps=eps, n_jobs=-1).fit(pc_colors)
    labels = clustering.labels_
    
    max_label = np.max(labels)
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    point_cloud = o3d.geometry.PointCloud(o3d.cpu.pybind.utility.Vector3dVector(point_cloud))
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    return point_cloud, labels

# Try k_means for fun
def k_means_color(point_cloud, pc_colors, n_clusters):
    
    point_cloud = np.asarray(point_cloud.points)
    clustering = KMeans(n_clusters=n_clusters, n_jobs=-1).fit(pc_colors)
    
    labels = clustering.labels_
    max_label = np.max(labels)
    print(f"point cloud has {max_label + 1} clusters")
    
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    point_cloud = o3d.geometry.PointCloud(o3d.cpu.pybind.utility.Vector3dVector(point_cloud))
    point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
    
    return point_cloud, labels


In [3]:
#pc = o3d.io.read_point_cloud('../../3D-data/cropped_pcd_smaller_downsampled_and_filtered_rad20.ply')
#pc = o3d.io.read_point_cloud('../../3D-data/cropped_pcd_filtered_rad20_0p1_nopot.ply')
#pc = o3d.io.read_point_cloud('../../3D-data/cropped_pcd_filtered_newplant.ply')
#pc = o3d.io.read_point_cloud('./cropped_pcd_filtered_newplant.ply')
#pc = o3d.io.read_point_cloud('../../3D-data/cropped_pcd_filtered_avocado_rad20_0p1.ply')
#pc = o3d.io.read_point_cloud('./cropped_pcd_filtered_avocado_rad20_0p1.ply')
pc = o3d.io.read_point_cloud('./cropped_pcd_filtered_rad20_0p1_nopot.ply')
pc = pc.uniform_down_sample(every_k_points=int(len(np.asarray(pc.points))/15000))
print(len(np.asarray(pc.points)))
#o3d.visualization.draw_geometries([pc], window_name='t')

20205


In [4]:
pc_colors = np.asarray(pc.colors) # RGB colors

pc_gray   = 0.299*pc_colors[:,0] + 0.587*pc_colors[:,1] + 0.114*pc_colors[:,2] 
pc_gray     = np.c_[np.asarray(pc.points), pc_gray] # grayscale cloud

for i in range(len(pc_colors)):
    pc_colors[i] = colorsys.rgb_to_hsv(pc_colors[i,0],pc_colors[i,1],pc_colors[i,2]) # Convert to hsv
pc_colors = np.c_[np.asarray(pc.points), pc_colors]  # color cloud

In [5]:
ls_cluster =[]
print(time.asctime())
for i in range(5):
    for j in range(6):
        print(i, j)
        ls_cluster.append(dbscan_color(pc, pc_colors, 10**(i-4), 2**j))
print(time.asctime())

Tue May  4 16:13:23 2021
0 0
clustering:  DBSCAN(eps=0.0001, min_samples=1, n_jobs=-1)
labels:  [    0     1     2 ... 20202 20203 20204]
point cloud has 20205 clusters
colors:  [[0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 ...
 [0.61960784 0.85490196 0.89803922 1.        ]
 [0.61960784 0.85490196 0.89803922 1.        ]
 [0.61960784 0.85490196 0.89803922 1.        ]]
0 1
clustering:  DBSCAN(eps=0.0001, min_samples=2, n_jobs=-1)
labels:  [-1 -1 -1 ... -1 -1 -1]
point cloud has 0 clusters
colors:  [[0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 ...
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]]
0 2
clustering:  DBSCAN(eps=0.0001, min_samples=4, n_jobs=-1)
labels:  [-1 -1 -1 ... -1 -1 -1]
point cloud has 0 cl

In [6]:
i=3
j=4.2
clustered_pc, labels = dbscan_color(pc, pc_colors, 10**(i-4), 2**j, False) # was 6D
o3d.visualization.draw_geometries([clustered_pc], window_name='t')

'''
i=3
j=5
clustered_pc, labels = dbscan_4D(pc, pc_4D, 10**(i-4), 2**j, False)
o3d.visualization.draw_geometries([clustered_pc], window_name='t')
'''

clustering:  DBSCAN(eps=0.1, min_samples=18.37917367995256, n_jobs=-1)
labels:  [ 1 -1 -1 ...  5  2  6]
point cloud has 22 clusters
colors:  [[0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 [0.12156863 0.46666667 0.70588235 1.        ]
 ...
 [0.17254902 0.62745098 0.17254902 1.        ]
 [0.68235294 0.78039216 0.90980392 1.        ]
 [0.59607843 0.8745098  0.54117647 1.        ]]


"\ni=3\nj=5\nclustered_pc, labels = dbscan_4D(pc, pc_4D, 10**(i-4), 2**j, False)\no3d.visualization.draw_geometries([clustered_pc], window_name='t')\n"

In [7]:
mask = [la==5 for la in labels]
print(np.asarray(pc.points))
points = np.asarray(pc.points)
print(points.shape)
#print(mask)

[[ 2.47904706  0.73929596  5.27709103]
 [ 2.58115005  0.64347792  5.25052261]
 [ 2.58010936  0.62552333  5.24653316]
 ...
 [ 3.09278202 -1.50414658  4.21751118]
 [ 0.70030665 -1.11335313  4.36267757]
 [ 1.29057574 -2.88004947  3.18366849]]
(20205, 3)


In [13]:
#print(points[mask])
selected = points[mask]
print(selected)
print(selected.shape)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(selected)

o3d.visualization.draw_geometries([pcd], window_name='t')

o3d.io.write_point_cloud("leaf-test.ply", pcd)

[[ 1.81668663 -0.67773604  2.74649405]
 [ 2.43406105 -1.04012847  3.29532599]
 [ 2.13472462 -1.01650572  3.04951   ]
 ...
 [ 2.63125873 -1.21123767  3.34649611]
 [ 3.08506816 -0.75395831  3.45663347]
 [ 3.09278202 -1.50414658  4.21751118]]
(4555, 3)


True

In [9]:
print(len(labels))
print(len(pc.points))

20205
20205


In [7]:
i=2
j=1
clustered_pc, labels = dbscan(pc, 10**(i-4), 2**j, False) # was 6D
o3d.visualization.draw_geometries([clustered_pc], window_name='t')


[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 723
point cloud has 723 clusters


In [8]:
# Ignore warnings and make the output less crowded. 
# We get warnings to increase max_eps or all data will be considered outliers
# It can be useful to see the warnings, but it's obvious anyway if there are 0 clusters
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    ls_cluster =[]
    print(time.asctime())
    for i in np.arange(2,20):
        for j in np.arange(2,6):
            print(i, j)
            print(time.asctime())
            ls_cluster.append(optics_color(pc, pc_colors, 10**((i*1.0)-4), 2**j))
    print(time.asctime())

Tue May  4 14:30:32 2021
2 2
Tue May  4 14:30:32 2021
point cloud has 0 clusters
2 3
Tue May  4 14:30:33 2021
point cloud has 0 clusters
2 4
Tue May  4 14:30:35 2021
point cloud has 0 clusters
2 5
Tue May  4 14:30:36 2021
point cloud has 0 clusters
3 2
Tue May  4 14:30:38 2021
point cloud has 1501 clusters
3 3
Tue May  4 14:31:25 2021
point cloud has 373 clusters
3 4
Tue May  4 14:32:09 2021


KeyboardInterrupt: 

In [46]:
i=17
j=4
clustered_pc, labels = optics_color(pc, pc_colors, 10**(i-4), 2**j, False)
o3d.visualization.draw_geometries([clustered_pc], window_name='t')

point cloud has 51 clusters


In [31]:
# Ignore warnings and make the output less crowded. 
# We get warnings to increase max_eps or all data will be considered outliers
# It can be useful to see the warnings, but it's obvious anyway if there are 0 clusters
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    ls_cluster =[]
    print(time.asctime())
    for i in np.arange(5,20):
        for j in np.arange(2,6):
            print(i, j)
            ls_cluster.append(optics_color(pc, pc_colors, 10**((i*1.0)-4), 2**j))
    print(time.asctime())

Sun Apr 25 15:37:57 2021
5 2
point cloud has 1486 clusters
5 3
point cloud has 316 clusters
5 4
point cloud has 71 clusters
5 5
point cloud has 8 clusters
6 2
point cloud has 1486 clusters
6 3
point cloud has 316 clusters
6 4
point cloud has 71 clusters
6 5
point cloud has 8 clusters
7 2
point cloud has 1486 clusters
7 3


KeyboardInterrupt: 

In [39]:
i=6
j=5
clustered_pc, labels = optics_color(pc, pc_colors, 10**(i-4), 2**j, False)
o3d.visualization.draw_geometries([clustered_pc], window_name='t')

point cloud has 8 clusters


In [12]:
# Try kmeans
clustered_pc, labels = k_means_color(pc, pc_colors, 6)
o3d.visualization.draw_geometries([clustered_pc], window_name='t')

point cloud has 6 clusters


In [1]:
## DBSCAN without color, if needed for comparison

In [6]:
ls_cluster =[]
print(time.asctime())
for i in range(5):
    for j in range(6):
        print(i, j)
        ls_cluster.append(dbscan(pc, 10**(i-4), 2**j))
print(time.asctime())

Sun Apr 25 14:37:07 2021
0 0
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 26828
point cloud has 26828 clusters
0 1
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
0 2
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
0 3
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
0 4
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 0
point cloud has 0 clusters
0 5
[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] D

In [8]:
i=2
j=2
clustered_pc = dbscan(pc, 10**(i-4), 2**j, False) # was 6D
o3d.visualization.draw_geometries([clustered_pc], window_name='t')

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 10
point cloud has 10 clusters


In [7]:
# TODO: Extract individual leaves and generate mesh 
# The code to generate the mesh is currently in src/outlier_removal.py
# Then everytging is ready I can write a function for each step, 
# which will make the final main file very tidy and readable